In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

In [8]:
# Loading the dataset and perform splitting
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Peforming reshaping operation
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Normalization
x_train = x_train / 255
x_test = x_test / 255

# One Hot Encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Base-LeNet

In [9]:
def build_lenet():
    model = Sequential()
    # Conv-1: Filter as we know 6, filter_size= 5 x 5, 'tanh' is the activation and input size is 28 X 28 grayscap images
    model.add(Conv2D(
        filters=6,
        kernel_size = (5,5),
        strides=(1,1),
        activation='tanh',
        input_shape=(28, 28, 1)
        )
    )
    # Subsampling-1: input for this layer (28 x 28 x 6) Output= (14 x 14 x 6)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Conv-2: input for this layer (14 x 14 x 6) Output= (10 x 10 x 16)
    model.add(Conv2D(
        filters=16,
        kernel_size = (5,5),
        strides=(1,1),
        activation='tanh'
        )
    )

    # Subsampling-2: input for this layer (10 x 10 x 16) Output= (5 x 5 x 16)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Flatten: here 5 * 5 * 16 matrix  pulled into a vector

    model.add(Flatten())
    # The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
    model.add(Dense(units=128, activation='tanh'))
    # The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
    model.add(Dense(units=84, activation='tanh'))
    # The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
    model.add(Dense(units=10, activation='softmax'))

    model.summary()
    model.compile(loss=tf.keras.metrics.categorical_crossentropy, optimizer=tf.keras.optimizers.SGD(lr=0.1), metrics=['accuracy'])
    return model

In [4]:
model_1 = build_lenet()

model_1.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))
score = model_1.evaluate(x_test, y_test, verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 6)         156       
                                                                 
 average_pooling2d (Average  (None, 12, 12, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          2416      
                                                                 
 average_pooling2d_1 (Avera  (None, 4, 4, 16)          0         
 gePooling2D)                                                    
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               3

Epoch 1/20
469/469 [==============================] - 26s 53ms/step - loss: 1.2091 - accuracy: 0.6939 - val_loss: 0.5905 - val_accuracy: 0.8601
Epoch 2/20
469/469 [==============================] - 25s 54ms/step - loss: 0.4986 - accuracy: 0.8698 - val_loss: 0.4124 - val_accuracy: 0.8895
Epoch 3/20
469/469 [==============================] - 24s 52ms/step - loss: 0.3932 - accuracy: 0.8902 - val_loss: 0.3482 - val_accuracy: 0.9034
Epoch 4/20
469/469 [==============================] - 23s 49ms/step - loss: 0.3437 - accuracy: 0.9021 - val_loss: 0.3105 - val_accuracy: 0.9120
Epoch 5/20
469/469 [==============================] - 25s 53ms/step - loss: 0.3102 - accuracy: 0.9098 - val_loss: 0.2831 - val_accuracy: 0.9190
Epoch 6/20
469/469 [==============================] - 25s 53ms/step - loss: 0.2838 - accuracy: 0.9173 - val_loss: 0.2605 - val_accuracy: 0.9236
Epoch 7/20
469/469 [==============================] - 24s 52ms/step - loss: 0.2619 - accuracy: 0.9233 - val_loss: 0.2403 - val_accuracy:

# Modified LeNet

In [5]:
def build_lenet_with_relu():
    # Building the Model Architecture

    model = Sequential()
    # Select 6 feature convolution kernels with a size of 5 * 5 (without offset), and get 66 feature maps. The size of each feature map is 32−5 + 1 = 2832−5 + 1 = 28.
    # That is, the number of neurons has been reduced from 10241024 to 28 ∗ 28 = 784 28 ∗ 28 = 784.
    # Parameters between input layer and C1 layer: 6 ∗ (5 ∗ 5 + 1)
    model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
    # The input of this layer is the output of the first layer, which is a 28 * 28 * 6 node matrix.
    # The size of the filter used in this layer is 2 * 2, and the step length and width are both 2, so the output matrix size of this layer is 14 * 14 * 6.
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # The input matrix size of this layer is 14 * 14 * 6, the filter size used is 5 * 5, and the depth is 16. This layer does not use all 0 padding, and the step size is 1.
    # The output matrix size of this layer is 10 * 10 * 16. This layer has 5 * 5 * 6 * 16 + 16 = 2416 parameters
    model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
    # The input matrix size of this layer is 10 * 10 * 16. The size of the filter used in this layer is 2 * 2, and the length and width steps are both 2, so the output matrix size of this layer is 5 * 5 * 16.
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # The input matrix size of this layer is 5 * 5 * 16. This layer is called a convolution layer in the LeNet-5 paper, but because the size of the filter is 5 * 5, #
    # So it is not different from the fully connected layer. If the nodes in the 5 * 5 * 16 matrix are pulled into a vector, then this layer is the same as the fully connected layer.
    # The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    # The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
    model.add(Dense(84, activation='relu'))
    # The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
    model.add(Dense(10, activation='softmax'))

    model.summary()
    model.compile(loss=tf.keras.metrics.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
    return model



In [6]:
# building the model
model_2 = build_lenet_with_relu()

model_2.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))
score = model_2.evaluate(x_test, y_test)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 24, 24, 6)         156       
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 6)         0         
 D)                                                              
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 16)          2416      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 4, 4, 16)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 120)              

In [11]:
def build_lenet_3_3():
    model = Sequential()
    # Conv-1: Filter as we know 6, filter_size= 5 x 5, 'tanh' is the activation and input size is 28 X 28 grayscap images
    model.add(Conv2D(
        filters=6,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh',
        input_shape=(28, 28, 1)
        )
    )
    # Subsampling-1: input for this layer (28 x 28 x 6) Output= (14 x 14 x 6)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Conv-2: input for this layer (14 x 14 x 6) Output= (10 x 10 x 16)
    model.add(Conv2D(
        filters=16,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )

    # Subsampling-2: input for this layer (10 x 10 x 16) Output= (5 x 5 x 16)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Flatten: here 5 * 5 * 16 matrix  pulled into a vector

    model.add(Flatten())
    # The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
    model.add(Dense(units=128, activation='tanh'))
    # The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
    model.add(Dense(units=84, activation='tanh'))
    # The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
    model.add(Dense(units=10, activation='softmax'))

    model.summary()
    model.compile(loss=tf.keras.metrics.categorical_crossentropy, optimizer=tf.keras.optimizers.SGD(lr=0.1), metrics=['accuracy'])
    return model

In [12]:
model_3 = build_lenet_3_3()

model_3.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))
score = model_3.evaluate(x_test, y_test, verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 6)         60        
                                                                 
 average_pooling2d_2 (Avera  (None, 13, 13, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 16)        880       
                                                                 
 average_pooling2d_3 (Avera  (None, 5, 5, 16)          0         
 gePooling2D)                                                    
                                                                 
 flatten_2 (Flatten)         (None, 400)               0         
                                                                 
 dense_6 (Dense)             (None, 128)              

Epoch 1/20
469/469 [==============================] - 21s 43ms/step - loss: 1.3456 - accuracy: 0.6519 - val_loss: 0.6431 - val_accuracy: 0.8510
Epoch 2/20
469/469 [==============================] - 19s 41ms/step - loss: 0.5269 - accuracy: 0.8658 - val_loss: 0.4333 - val_accuracy: 0.8834
Epoch 3/20
469/469 [==============================] - 23s 48ms/step - loss: 0.4073 - accuracy: 0.8872 - val_loss: 0.3631 - val_accuracy: 0.8969
Epoch 4/20
469/469 [==============================] - 20s 42ms/step - loss: 0.3550 - accuracy: 0.8985 - val_loss: 0.3233 - val_accuracy: 0.9050
Epoch 5/20
469/469 [==============================] - 20s 43ms/step - loss: 0.3223 - accuracy: 0.9063 - val_loss: 0.2946 - val_accuracy: 0.9144
Epoch 6/20
469/469 [==============================] - 18s 39ms/step - loss: 0.2977 - accuracy: 0.9128 - val_loss: 0.2727 - val_accuracy: 0.9197
Epoch 7/20
469/469 [==============================] - 19s 41ms/step - loss: 0.2777 - accuracy: 0.9187 - val_loss: 0.2551 - val_accuracy:

In [13]:
def build_lenet_3_3_2():
    model = Sequential()
    # Conv-1: Filter as we know 6, filter_size= 5 x 5, 'tanh' is the activation and input size is 28 X 28 grayscap images
    model.add(Conv2D(
        filters=6,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh',
        input_shape=(28, 28, 1)
        )
    )

    model.add(Conv2D(
        filters=6,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh',
        input_shape=(28, 28, 1)
        )
    )
    # Subsampling-1: input for this layer (28 x 28 x 6) Output= (14 x 14 x 6)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Conv-2: input for this layer (14 x 14 x 6) Output= (10 x 10 x 16)
    model.add(Conv2D(
        filters=16,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )
    model.add(Conv2D(
        filters=16,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )

    # Subsampling-2: input for this layer (10 x 10 x 16) Output= (5 x 5 x 16)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Flatten: here 5 * 5 * 16 matrix  pulled into a vector

    model.add(Flatten())
    # The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
    model.add(Dense(units=128, activation='tanh'))
    # The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
    model.add(Dense(units=84, activation='tanh'))
    # The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
    model.add(Dense(units=10, activation='softmax'))

    model.summary()
    model.compile(loss=tf.keras.metrics.categorical_crossentropy, optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), metrics=['accuracy'])
    return model

# Building model
model_4 = build_lenet_3_3_2()

model_4.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))
score = model_4.evaluate(x_test, y_test, verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 6)         60        
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 6)         330       
                                                                 
 average_pooling2d_4 (Avera  (None, 12, 12, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_8 (Conv2D)           (None, 10, 10, 16)        880       
                                                                 
 conv2d_9 (Conv2D)           (None, 8, 8, 16)          2320      
                                                                 
 average_pooling2d_5 (Avera  (None, 4, 4, 16)          0         
 gePooling2D)                                         

# More filters

In [14]:
def build_lenet_filler_1():
    model = Sequential()
    # Conv-1: Filter as we know 6, filter_size= 5 x 5, 'tanh' is the activation and input size is 28 X 28 grayscap images
    model.add(Conv2D(
        filters=8,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh',
        input_shape=(28, 28, 1)
        )
    )
    model.add(Conv2D(
        filters=8,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh',
        input_shape=(28, 28, 1)
        )
    )

    # Subsampling-1: input for this layer (28 x 28 x 6) Output= (14 x 14 x 6)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Conv-2: input for this layer (14 x 14 x 6) Output= (10 x 10 x 16)
    model.add(Conv2D(
        filters=16,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )

    model.add(Conv2D(
        filters=32,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )

    # Subsampling-2: input for this layer (10 x 10 x 16) Output= (5 x 5 x 16)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Flatten: here 5 * 5 * 16 matrix  pulled into a vector

    model.add(Flatten())
    # The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
    model.add(Dense(units=128, activation='tanh'))
    # The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
    model.add(Dense(units=84, activation='tanh'))
    # The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
    model.add(Dense(units=10, activation='softmax'))

    model.summary()
    model.compile(loss=tf.keras.metrics.categorical_crossentropy, optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), metrics=['accuracy'])
    return model

# Building model
model_5 = build_lenet_filler_1()

model_5.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))
score = model_5.evaluate(x_test, y_test, verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 26, 26, 8)         80        
                                                                 
 conv2d_11 (Conv2D)          (None, 24, 24, 8)         584       
                                                                 
 average_pooling2d_6 (Avera  (None, 12, 12, 8)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_12 (Conv2D)          (None, 10, 10, 16)        1168      
                                                                 
 conv2d_13 (Conv2D)          (None, 8, 8, 32)          4640      
                                                                 
 average_pooling2d_7 (Avera  (None, 4, 4, 32)          0         
 gePooling2D)                                         

In [15]:
def build_lenet_filler_1():
    model = Sequential()
    # Conv-1: Filter as we know 6, filter_size= 5 x 5, 'tanh' is the activation and input size is 28 X 28 grayscap images
    model.add(Conv2D(
        filters=8,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh',
        input_shape=(28, 28, 1)
        )
    )
    model.add(Conv2D(
        filters=8,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh',
        input_shape=(28, 28, 1)
        )
    )

    # Subsampling-1: input for this layer (28 x 28 x 6) Output= (14 x 14 x 6)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Conv-2: input for this layer (14 x 14 x 6) Output= (10 x 10 x 16)
    model.add(Conv2D(
        filters=16,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )
    model.add(Conv2D(
        filters=16,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )

    model.add(Conv2D(
        filters=32,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )
    model.add(Conv2D(
        filters=32,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )

    # Subsampling-2: input for this layer (10 x 10 x 16) Output= (5 x 5 x 16)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Flatten: here 5 * 5 * 16 matrix  pulled into a vector

    model.add(Flatten())
    # The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
    model.add(Dense(units=128, activation='tanh'))
    # The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
    model.add(Dense(units=84, activation='tanh'))
    # The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
    model.add(Dense(units=10, activation='softmax'))

    model.summary()
    model.compile(loss=tf.keras.metrics.categorical_crossentropy, optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), metrics=['accuracy'])
    return model

# Building model
model_5 = build_lenet_filler_1()

model_5.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))
score = model_5.evaluate(x_test, y_test, verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 26, 26, 8)         80        
                                                                 
 conv2d_15 (Conv2D)          (None, 24, 24, 8)         584       
                                                                 
 average_pooling2d_8 (Avera  (None, 12, 12, 8)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_16 (Conv2D)          (None, 10, 10, 16)        1168      
                                                                 
 conv2d_17 (Conv2D)          (None, 8, 8, 16)          2320      
                                                                 
 conv2d_18 (Conv2D)          (None, 6, 6, 32)          4640      
                                                      

# More dense Unit

In [ ]:
def build_lenet_more_dense():
    model = Sequential()
    # Conv-1: Filter as we know 6, filter_size= 5 x 5, 'tanh' is the activation and input size is 28 X 28 grayscap images
    model.add(Conv2D(
        filters=6,
        kernel_size = (5,5),
        strides=(1,1),
        activation='tanh',
        input_shape=(28, 28, 1)
        )
    )
    # Subsampling-1: input for this layer (28 x 28 x 6) Output= (14 x 14 x 6)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Conv-2: input for this layer (14 x 14 x 6) Output= (10 x 10 x 16)
    model.add(Conv2D(
        filters=16,
        kernel_size = (5,5),
        strides=(1,1),
        activation='tanh'
        )
    )

    # Subsampling-2: input for this layer (10 x 10 x 16) Output= (5 x 5 x 16)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Flatten: here 5 * 5 * 16 matrix  pulled into a vector

    model.add(Flatten())
    # The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
    model.add(Dense(units=128, activation='tanh'))
    # The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
    model.add(Dense(units=64, activation='tanh'))
    model.add(Dense(units=32, activation='tanh'))
    model.add(Dense(units=16, activation='tanh'))
    # The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
    model.add(Dense(units=10, activation='softmax'))

    model.summary()
    model.compile(loss=tf.keras.metrics.categorical_crossentropy, optimizer=tf.keras.optimizers.SGD(lr=0.1), metrics=['accuracy'])
    return model


# Building model
model_6 = build_lenet_filler_1()

model_6.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))
score = model_6.evaluate(x_test, y_test, verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 26, 26, 8)         80        
                                                                 
 conv2d_21 (Conv2D)          (None, 24, 24, 8)         584       
                                                                 
 average_pooling2d_10 (Aver  (None, 12, 12, 8)         0         
 agePooling2D)                                                   
                                                                 
 conv2d_22 (Conv2D)          (None, 10, 10, 16)        1168      
                                                                 
 conv2d_23 (Conv2D)          (None, 8, 8, 16)          2320      
                                                                 
 conv2d_24 (Conv2D)          (None, 6, 6, 32)          4640      
                                                      

In [17]:
def build_lenet_filler_1():
    model = Sequential()
    # Conv-1: Filter as we know 6, filter_size= 5 x 5, 'tanh' is the activation and input size is 28 X 28 grayscap images
    model.add(Conv2D(
        filters=8,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh',
        input_shape=(28, 28, 1)
        )
    )
    model.add(Conv2D(
        filters=8,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh',
        input_shape=(28, 28, 1)
        )
    )

    # Subsampling-1: input for this layer (28 x 28 x 6) Output= (14 x 14 x 6)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Conv-2: input for this layer (14 x 14 x 6) Output= (10 x 10 x 16)
    model.add(Conv2D(
        filters=16,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )
    model.add(Conv2D(
        filters=16,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )

    model.add(Conv2D(
        filters=32,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )
    model.add(Conv2D(
        filters=32,
        kernel_size = (3,3),
        strides=(1,1),
        activation='tanh'
        )
    )

    # Subsampling-2: input for this layer (10 x 10 x 16) Output= (5 x 5 x 16)
    model.add(
        AveragePooling2D(
            pool_size=(2,2),
            strides=(2,2)
        )
    )
    # Flatten: here 5 * 5 * 16 matrix  pulled into a vector

    model.add(Flatten())
    # The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
    model.add(Dense(units=128, activation='tanh'))
    # The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
    model.add(Dense(units=64, activation='tanh'))
    model.add(Dense(units=32, activation='tanh'))
    model.add(Dense(units=12, activation='tanh'))
    # The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
    model.add(Dense(units=10, activation='softmax'))

    model.summary()
    model.compile(loss=tf.keras.metrics.categorical_crossentropy, optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), metrics=['accuracy'])
    return model

# Building model
model_7 = build_lenet_filler_1()

model_7.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))
score = model_7.evaluate(x_test, y_test, verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 26, 26, 8)         80        
                                                                 
 conv2d_27 (Conv2D)          (None, 24, 24, 8)         584       
                                                                 
 average_pooling2d_12 (Aver  (None, 12, 12, 8)         0         
 agePooling2D)                                                   
                                                                 
 conv2d_28 (Conv2D)          (None, 10, 10, 16)        1168      
                                                                 
 conv2d_29 (Conv2D)          (None, 8, 8, 16)          2320      
                                                                 
 conv2d_30 (Conv2D)          (None, 6, 6, 32)          4640      
                                                      